In [15]:
%load_ext tensorboard

In [1]:
import numpy as np
import tensorflow as tf
import librosa
import pandas as pd
import os

In [2]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

In [ ]:
tf.__version__

In [ ]:
#SES02 as test remain all train

In [ ]:
tf=pd.read_csv('Audio_links/Ses02.csv')


In [ ]:
tf.shape

In [ ]:
df=pd.concat([df,pf],axis=0)

In [52]:
df.sample(10)

,Unnamed: 0,name,Emot,file_path
461,461,Ses01F_impro04_M013,neutral,E:/Work/Audio DATA/IEMOCAP_full_release_withou...
6671,6671,Ses05M_script03_2_F002,neutral,E:/Work/Audio DATA/IEMOCAP_full_release_withou...
2980,2980,Ses05F_script02_2_M010,neutral,E:/Work/Audio DATA/IEMOCAP_full_release_withou...
6314,6314,Ses05M_script03_1_F031,happy,E:/Work/Audio DATA/IEMOCAP_full_release_withou...
1111,1111,Ses01F_impro07_M010,neutral,E:/Work/Audio DATA/IEMOCAP_full_release_withou...
1535,1535,Ses01F_script01_3_F018,happy,E:/Work/Audio DATA/IEMOCAP_full_release_withou...
3828,3828,Ses05M_impro02_M017,angry,E:/Work/Audio DATA/IEMOCAP_full_release_withou...
283,283,Ses01F_impro03_M009,happy,E:/Work/Audio DATA/IEMOCAP_full_release_withou...
941,941,Ses05F_impro04_F043,neutral,E:/Work/Audio DATA/IEMOCAP_full_release_withou...
86,86,Ses05F_impro01_M013,angry,E:/Work/Audio DATA/IEMOCAP_full_release_withou...


In [ ]:
df.shape

In [ ]:
dataset_dict = {
    'Emotion_id':{ 0:'neutral', 
            1 : 'happy', 
            2: 'sad', 
            3: 'angry', 
            4: 'fearful' ,
            5 :'disgust'}
    ,
    'gender_id': {
        0: 'Male',
        1: 'Female'
    }
}

dataset_dict['gender_alias'] = dict((g, i) for i, g in dataset_dict['gender_id'].items())
dataset_dict['Emotion_alias'] = dict((g, i) for i, g in dataset_dict['Emotion_id'].items())

In [ ]:
#df['gender_id']= df['gender'].map(lambda gender: dataset_dict['gender_alias'][gender])
df['Label_id']= df['Emot'].map(lambda race: dataset_dict['Emotion_alias'][race])
        

In [ ]:
df.sample(10)

In [ ]:
from sklearn.model_selection import train_test_split
trainin,valind=train_test_split(df,test_size=0.15,random_state=20)

In [ ]:
# Feature extraction of  a file
def fearture_extrack(file_path):
    SAMPLE_RATE = 22050
    
    signal,sr=librosa.load(file_path,sr=SAMPLE_RATE)
    
    if len(signal) >= SAMPLES_TO_CONSIDER:

                    # ensure consistency of the length of the signal
            signal = signal[:SAMPLES_TO_CONSIDER]

                    # extract MFCC
            MFCCs = librosa.feature.mfcc(signal,sr,n_mfcc=128,n_fft=2048,hop_length=512).T
            return MFCCs
    

In [ ]:
# start find max lenght
def padding_n_reshape(audio_list):
    l=[]

    for i in range(len(audio_list)):
        l.append(audio_list[i].shape[0])
    maxy=max(l)
    #print(maxy)

    #padding

    npp=[]
    for i in audio_list:
        result = np.zeros((maxy,18))
        result[:i.shape[0],:i.shape[1]] = i
        npp.append(result)

    #retutn tensor
    newarr=np.array(npp)

    new=np.reshape(newarr,(newarr.shape[0],newarr.shape[1],newarr.shape[2],1))
    
    return new

In [ ]:
# genrator for data passing into model traning
from tensorflow.keras.utils import to_categorical
from tensorflow import convert_to_tensor
def return_data(data_frame):

    filea,label=[],[]#filea,age,sex,label=[],[],[],[]
    
    for inx,row in data_frame.iterrows():
    
        #ag=row['Age']
        #gen=row['gender_id']
        emot=row['Label_id']
        file=row['file_path']
        

        
        SAMPLES_TO_CONSIDER = 22050*3
    
        signal,sr=librosa.load(file)
    
        if len(signal) >= SAMPLES_TO_CONSIDER:

            # ensure consistency of the length of the signal
            signal = signal[:SAMPLES_TO_CONSIDER]

            # extract MFCC
            MFCCs = librosa.feature.mfcc(signal,sr,n_mfcc=128,n_fft=2048,hop_length=512).T
            filea.append(MFCCs)
            label.append(to_categorical(emot,len(df.Emot.unique())))


 
    return np.array(filea),np.array(label,dtype=np.float32)#np.array(age,dtype=np.float32),np.array(sex,dtype=np.float32),
    #filea,age,sex,label=[],[],[],[]#filea,label=[],[]

In [ ]:
train=return_data(trainin)


In [ ]:
train[0][245].shape

In [ ]:
#save
import pickle 
with open('audio_arr/TrainXall.pkl', 'wb') as f:
    pickle.dump(train,f)

In [1]:
#load
import pickle 
with open('audio_arr/trainCrema.pkl', 'rb') as f: 
    train = pickle.load(f)

In [2]:
train[0].shape

(1116, 130, 128)

In [10]:
trainX=train[0]
trainy=train[1]

In [59]:
trainy

array([[0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       ...,
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]], dtype=float32)

In [12]:
trainX=np.reshape(trainX,(trainX.shape[0],trainX.shape[1],trainX.shape[2],1))

In [13]:
trainX.shape

(3003, 130, 128, 1)

In [ ]:
val=return_data(valind)

In [ ]:
#save
import pickle 
with open('audio_arr/ValXall.pkl', 'wb') as f:
    pickle.dump(val,f)

In [3]:
#load
import pickle 
with open('audio_arr/valCreama.pkl', 'rb') as f: 
    val = pickle.load(f)

In [4]:
val[0].shape

(136, 130, 128)

In [ ]:
val_X=val[0]
val_y=val[1]

In [ ]:
val_X=np.reshape(val_X,(val_X.shape[0],val_X.shape[1],val_X.shape[2],1))

In [ ]:
val_X.shape

In [ ]:
val_y

In [3]:
import datetime
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

In [ ]:
num_classes=6
mdl=Sequential()
mdl.add(Conv2D(96,padding= 'valid',kernel_size=(20,), activation= 'relu',
                        input_shape= (130,128,1),
                        kernel_initializer= 'he_normal'))
mdl.add(MaxPooling2D(pool_size=(20,3),strides=2))

mdl.add(MaxPooling2D(pool_size=(15,3),strides=(2,2)))
mdl.add(Conv2D(384, kernel_size=(9,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))
mdl.add(Conv2D(384, kernel_size=(9,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

mdl.add(Conv2D(256, kernel_size=(9,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))
mdl.add(MaxPooling2D(pool_size=(9,2),strides=(2,2)))
       

        
mdl.add(Flatten())
mdl.add(Dense(100, activation="relu", kernel_initializer="uniform"))
mdl.add(Dropout(0.5))
mdl.add(Dense(50, activation="relu", kernel_initializer="uniform"))
mdl.add(Dropout(0.5))
mdl.add(Dense(20, activation="relu", kernel_initializer="uniform"))
mdl.add(Dropout(0.5))
mdl.add(Dense(num_classes, activation= 'softmax',kernel_initializer="uniform"))


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
chk_filepath = './checkpoint/'

chk_point=ModelCheckpoint(chk_filepath,
                          save_best_only=True,
                         monitor='accuracy',
                         verbose=0,
                         mode='max',
                         save_freq=5

                        )

In [ ]:
early_stop=EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1,
    mode='min',
    restore_best_weights=True
                        )

In [ ]:
num_classes=6
CNN_model=Sequential()

CNN_model.add(Conv2D(32,(3,3),input_shape=(130,128,1),padding='same'))
CNN_model.add(Conv2D(32,(3,3),padding='same'))
CNN_model.add(MaxPooling2D(pool_size=(2,2),strides=2))
CNN_model.add(Conv2D(32,(3,3),padding='same'))
CNN_model.add(Conv2D(32,(3,3),padding='same'))
CNN_model.add(MaxPooling2D(pool_size=(4,4),strides=4))

CNN_model.add(Flatten())
CNN_model.add(Dense(100, activation="relu", kernel_initializer="uniform"))
CNN_model.add(Dropout(0.5))
CNN_model.add(Dense(50, activation="relu", kernel_initializer="uniform"))
CNN_model.add(Dropout(0.5))
CNN_model.add(Dense(20, activation="relu", kernel_initializer="uniform"))
CNN_model.add(Dropout(0.5))
CNN_model.add(Dense(num_classes, activation= 'softmax',kernel_initializer="uniform"))

In [ ]:
CNN_model.compile(optimizer= tf.keras.optimizers.Adam(0.001),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])


In [4]:
import h5py

In [5]:
h5py.__version__

'2.10.0'

In [6]:
from tensorflow.keras.models import load_model

In [7]:
modelN1=load_model("CNN_LSTM.h5")

In [8]:
modelN1.weights_

AttributeError: 'Sequential' object has no attribute 'weights_'

In [ ]:
CNN_model.summary()


In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
CNN_model.fit(trainX,trainy,
        batch_size=32,
        epochs=100,
        validation_data=(val_X,val_y),
        callbacks=[tensorboard_callback,chk_point],verbose=1)

In [18]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 12584), started 0:01:05 ago. (Use '!kill 12584' to kill it.)

In [ ]:
CNN_model.save('CNN_LSTM.h5')

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
pred=modelN1.predict(trainX,callbacks=[tensorboard_callback])

In [22]:
pred

array([[2.6399311e-01, 1.0128853e-01, 1.5124913e-01, 3.8764462e-01,
        3.7815582e-02, 5.8009081e-02],
       [4.0139788e-01, 2.6721215e-01, 9.5374724e-03, 2.3621491e-01,
        4.4195790e-02, 4.1441783e-02],
       [4.6068770e-01, 3.5666844e-01, 5.4030544e-03, 1.0951181e-01,
        3.8231540e-02, 2.9497411e-02],
       ...,
       [1.4776810e-03, 9.9852222e-01, 0.0000000e+00, 1.9732924e-16,
        1.5447655e-07, 3.7133689e-13],
       [4.7330204e-03, 9.9526536e-01, 1.9111202e-37, 9.7319430e-14,
        1.6680705e-06, 4.0817832e-11],
       [4.3469611e-01, 3.0303654e-01, 2.0614337e-02, 1.6258398e-01,
        4.0405378e-02, 3.8663656e-02]], dtype=float32)

In [49]:
trainy

array([[0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       ...,
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]], dtype=float32)

In [62]:
class_names = [ 'neutral', 'happy', 'sad', 'angry', 'fearful' ,'disgust']
TEST_CLASS=[]
for i in range(len(trainy)):
    TEST_CLASS.append(class_names[np.argmax(trainy[i])])

In [63]:
TEST_CLASS

['happy',
 'angry',
 'angry',
 'angry',
 'angry',
 'happy',
 'neutral',
 'sad',
 'neutral',
 'angry',
 'neutral',
 'angry',
 'neutral',
 'angry',
 'angry',
 'happy',
 'sad',
 'angry',
 'happy',
 'angry',
 'angry',
 'sad',
 'angry',
 'happy',
 'angry',
 'angry',
 'neutral',
 'angry',
 'neutral',
 'neutral',
 'neutral',
 'happy',
 'happy',
 'happy',
 'neutral',
 'angry',
 'angry',
 'angry',
 'sad',
 'neutral',
 'happy',
 'happy',
 'angry',
 'sad',
 'neutral',
 'neutral',
 'happy',
 'angry',
 'angry',
 'sad',
 'happy',
 'angry',
 'sad',
 'angry',
 'neutral',
 'sad',
 'happy',
 'angry',
 'angry',
 'angry',
 'sad',
 'happy',
 'happy',
 'sad',
 'sad',
 'sad',
 'angry',
 'sad',
 'neutral',
 'angry',
 'happy',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'disgust',
 'happy',
 'neutral',
 'happy',
 'neutral',
 'angry',
 'angry',
 'happy',
 'sad',
 'neutral',
 'sad',
 'angry',
 'angry',
 'neutral',
 'angry',
 'neutral',
 'angry',
 'sad',
 'sad',
 'neutral',
 'angry',
 'sad',
 'happy',
 'angr

In [37]:
unique, counts = np.unique(PRED_CLASS, return_counts=True)

print(np.asarray((unique, counts)).T)

[['angry' '624']
 ['happy' '503']
 ['neutral' '1113']
 ['sad' '763']]


In [64]:
unique, counts = np.unique(TEST_CLASS, return_counts=True)

print(np.asarray((unique, counts)).T)

[['angry' '994']
 ['disgust' '69']
 ['fearful' '65']
 ['happy' '843']
 ['neutral' '640']
 ['sad' '392']]
